## Assignment 3 - Dimensionality Reduction
### Miguel Morales

This assignment is based on content discussed in module 6 and will work with the famous MNIST dataset, which is a set of images of handwritten digits https://en.wikipedia.org/wiki/MNIST_database.
The dataset has been provided to you in a .csv file.

## Learning outcomes

- Apply a Random Forest classification algorithm to MNIST dataset
- Perform dimensionality reduction of features using PCA and compare classification on the reduced dataset to that of original one
- Apply dimensionality reduction techniques: t-SNE and LLE

In [2]:
# Library imports for the assignment
import pandas as pd
import numpy as np

## Questions (15 points total)

__Question 1 (1 point).__ Load the MNIST dataset and split it into a training set and a test set (take the first 60,000 instances for training, and the remaining 10,000 for testing).

In [ ]:
# Analyzing the MNIST dataset (from the description on the Wikipedia page), and looking into its structure, we can determine the following:
# Columns:
# - Index column (unnamed): Row numbers 0-69,999
# - label column: The target variable (y): it contains the actual digit (0-9) that each image represents
# - 784 pixel columns: Named as 1x1, 1x2, 1x3, ..., 28x28
# -- These represent pixel intensity values from a 28×28 grayscale image (28*28 = 784)
# -- Each pixel value ranges from 0 (black) to 255 (white)
# -- The naming convention rowXcolumn indicates the position in the 28×28 grid

# Loading the MNIST dataset
mnist = pd.read_csv('../data/mnist_dataset.csv', index_col=0)

# Separate features (X) and labels (y)
X = mnist.drop('label', axis=1)
y = mnist['label']

# Splitting into training (first 60,000) and test (remaining 10,000) sets

# Training set: first 60,000 samples
X_train = X[:60000]
y_train = y[:60000]
# Test set: remaining 10,000 samples
X_test = X[60000:]
y_test = y[60000:]

# Displaying the shapes to verify the split
print(f"Training set (X_train): {X_train.shape[0]} samples")
print(f"Test set (X_test): {X_test.shape[0]} samples")
print(f"Number of features: {X_train.shape[1]}")
print(f"Labels in training set (the digits 0-9): {y_train.unique()}")


Training set (X_train): 60000 samples
Test set (X_test): 10000 samples
Number of features: 784
Labels in training set (the digits): [2 5 8 0 4 6 7 1 9 3]


__Question 2 (2 points).__ Train a Random Forest classifier on the dataset and time how long it takes, then evaluate the resulting model on the test set.

In [ ]:
# YOUR CODE HERE

__Question 3 (4 points).__ Next, use PCA to reduce the dataset’s dimensionality, with an explained variance ratio of 95%. Train a new Random Forest classifier on the reduced dataset and see how long it takes. Was training much faster? Next evaluate the classifier on the test set: how does it compare to the previous classifier?

In [ ]:
# YOUR CODE HERE

__Question 4 (4 points).__ Use t-SNE to reduce the MNIST dataset, show result graphically.

In [ ]:
# YOUR CODE HERE

__Question 5 (4 points).__ Compare with other dimensionality methods: _Locally Linear Embedding_ (LLE) or _Multidimensional scaling_ (MDS).

In [ ]:
# YOR CODE HERE

In [1]:
# -- Assignment 3 -- Dimensionality Reduction

# This assignment is based on content discussed in module 6 and will work with the famous MNIST dataset, which is a set of images of handwritten digits https://en.wikipedia.org/wiki/MNIST_database.
# The dataset has been provided to you in a .csv file.

## Learning outcomes

# - Apply a Random Forest classification algorithm to MNIST dataset
# - Perform dimensionality reduction of features using PCA and compare classification on the reduced dataset to that of original one
# - Apply dimensionality reduction techniques: t-SNE and LLE

## Questions (15 points total)

#__Question 1 (1 point).__ Load the MNIST dataset and split it into a training set and a test set (take the first 60,000 instances for training,
# and the remaining 10,000 for testing).

#__Question 2 (2 points).__ Train a Random Forest classifier on the dataset and time how long it takes, 
#then evaluate the resulting model on the test set.

#__Question 3 (4 points).__ Next, use PCA to reduce the dataset’s dimensionality, with an explained variance ratio of 95%. 
#Train a new Random Forest classifier on the reduced dataset and see how long it takes. Was training much faster?
#Next evaluate the classifier on the test set: how does it compare to the previous classifier?

#__Question 4 (4 points).__ Use t-SNE to reduce the MNIST dataset, show result graphically.